In [21]:
import json
import random

In [22]:
# Some cards are missing the cardSetId, I believe because they are original cards that were never put into legacy/classic/core
def fix_missing_set_id(all_cards_json):
	for card_name in all_cards_json:
			card_data = all_cards_json[card_name]
			if "cardSetId" not in card_data:
				card_data["cardSetId"] = 1635
				continue

In [23]:
def change_rarity_order(all_cards_json):
	for card_name in all_cards_json:
		card_data = all_cards_json[card_name]
		#Swaps the position of Common (1) and Free (2)
		if card_data["rarityId"] == 2: 
			card_data["rarityId"] = 1
		elif card_data["rarityId"] == 1:
			card_data["rarityId"] = 2

In [24]:
# Swaps free and common
def change_rarity_conversions(conversion_data_json):
	conversion_data_json["rarityId"]["1"] = "Free"
	conversion_data_json["rarityId"]["2"] = "Common"

In [25]:
def setup_set_name(conversion_data_json):
	conversion_data_json["cardSetId"] = conversion_data_json["sets"]
	del conversion_data_json["sets"]

In [26]:
def remove_unncessary_headers(columnHeaders, all_cards_json):
	output_dict = {}
	all_card_names = list(all_cards_json.keys())
	random.shuffle(all_card_names)
	collectible = set()
	for key in all_card_names:
		card_data = all_cards_json[key]
		name = card_data["name"].upper()
		output_dict[name] = {}
		keys = card_data.keys()
		for header in columnHeaders:
			if header in keys:
				if header == "cardSetId" and card_data[header] == 3:
					card_data[header] = 1635
				else:
					output_dict[name][header] = card_data[header]
	return output_dict

In [27]:
all_cards_unfiltered = json.loads(open("all_cards.txt", "r").read())
conversion_data = json.loads(open("id_conversions.json", "r").read())
fields = [
    "id",
    "collectible",
    "classId",
    "multiClassIds",
    "spellSchoolId",
    "cardTypeId",
    "cardSetId",
    "rarityId",
    "manaCost",
	"name",
    "image",
    "flavorText",
    "keywordIds",
]
all_cards_filtered = remove_unncessary_headers(fields, all_cards_unfiltered)
fix_missing_set_id(all_cards_filtered)
change_rarity_order(all_cards_filtered)
change_rarity_conversions(conversion_data)
setup_set_name(conversion_data)
open("all_cards.json", "w").write(json.dumps(all_cards_filtered, indent=4))
open("id_conversions.json", "w").write(json.dumps(conversion_data, indent=4))


4815